<a href="https://colab.research.google.com/github/shabnamsattar/Human-Data-Analysis/blob/main/chestMNIST/chestMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

# Uploading Dataset

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path1 = '/content/drive/My Drive/chestMNIST/chestmnist_64.npz'
path2 = '/content/drive/My Drive/chestMNIST/chestmnist_128.npz'
path3 = '/content/drive/My Drive/chestMNIST/chestmnist_224.npz'

## ChestMNIST_64

In [4]:
data_64 = np.load(path1, mmap_mode="r")

train_images_64 = data_64["train_images"]
train_labels_64 = data_64["train_labels"]
val_images_64   = data_64["val_images"]
val_labels_64   = data_64["val_labels"]
test_images_64  = data_64["test_images"]
test_labels_64  = data_64["test_labels"]


In [7]:
seed = 42
np.random.seed(seed)
percentage = 0.1

n1 = train_images_64.shape[0]
n2 = val_images_64.shape[0]
n3 = test_images_64.shape[0]

num_samples1 = int(n1 * percentage)
random_indices1 = np.random.choice(n1, num_samples1, replace=False)
num_samples2 = int(n2 * percentage)
random_indices2 = np.random.choice(n2, num_samples2, replace=False)
num_samples3 = int(n3 * percentage)
random_indices3 = np.random.choice(n3, num_samples3, replace=False)

train_images_64 = train_images_64[random_indices1]
train_labels_64 = train_labels_64[random_indices1]
val_images_64 = val_images_64[random_indices2]
val_labels_64 = val_labels_64[random_indices2]
test_images_64 = test_images_64[random_indices3]
test_labels_64 = test_labels_64[random_indices3]

In [13]:
from numpy.random import default_rng

rng = default_rng(42)
percentage = 0.1

def subsample(imgs, labels, pct=percentage):
    n = imgs.shape[0]
    m = int(n * pct)
    idx = rng.choice(n, size=m, replace=False)
    return imgs[idx], labels[idx]

train_images_64, train_labels_64 = subsample(train_images_64, train_labels_64)
val_images_64,   val_labels_64   = subsample(val_images_64,   val_labels_64)
test_images_64,  test_labels_64  = subsample(test_images_64,  test_labels_64)


## ChestMNIST_128

In [5]:
data_128 = np.load(path2, mmap_mode="r")

train_images_128 = data_128["train_images"]
train_labels_128 = data_128["train_labels"]
val_images_128   = data_128["val_images"]
val_labels_128   = data_128["val_labels"]
test_images_128  = data_128["test_images"]
test_labels_128  = data_128["test_labels"]


## ChestMNIST_224

In [6]:
data_224 = np.load(path3, mmap_mode="r")

train_images_224 = data_224["train_images"]
train_labels_224 = data_224["train_labels"]
val_images_224   = data_224["val_images"]
val_labels_224   = data_224["val_labels"]
test_images_224  = data_224["test_images"]
test_labels_224  = data_224["test_labels"]
